# Mid-term Exam (100 points)

This exam will use fitbit tweets I have collected between April 2019 and October 2019 excpet August 2019.

Please answer the following five questions using pyspark. **Please try to visualize your results whenever possible. After you are done, export your file as HTML, save it as a zip file and upload it on Blackboard.**

1. What is the number of retweets by week day? Which day has the most retweets?
2. For all tweets in Japanese (ja), what are the top 5 locations of the users excluding null and Japan.
3. Find the total number of tweets where the text contains either charge 2 or charge 3 (please check page 96 of the your text for references)
4. What is the percentage of verified users?
5. What is average rating of tweets between verified users and non-verified users?

Read the fitbit tweets, select fields to do furhter analysis. Please examine the fields carefully as I have imported additional fields for this exam.

**To speed up the processing time, we will only import the tweets in October 2019.**

In [0]:
# read tweets into a DataFrame

tweets=spark.read.json('/FileStore/tables/tweets/fitbit/2019_10.json')

# Select the fields that are of interest to do further analysis.
from pyspark.sql.functions import col, to_timestamp
tweets_selected=tweets.select(col('created_at').alias('date'), 'lang', 'source', col('id_str').alias('tweet_id'), col('user.screen_name').alias('user_name'), col('user.lang').alias('user_lang'), col('user.location').alias('user_location'), col('user.verified').alias('user_verified'), col('user.followers_count').alias('user_followers'), col('user.friends_count').alias('user_friends'), col('user.created_at').alias('user_joinDate'),col('retweeted_status.user.screen_name').alias('retweet_user'),
col('retweeted_status.reply_count').alias('reply_count'),
col('retweeted_status.retweet_count').alias('retweet_count'),
col('retweeted_status.favorite_count').alias('favorite_count'), 
col('retweeted_status.text').alias('retweet_text'),'text')


# convert tweet date, user joining date from string to timestamp
tweets_selected=tweets_selected.withColumn('date', to_timestamp('date','E MMM dd HH:mm:ss +0000 yyyy'))
tweets_selected=tweets_selected.withColumn('user_joinDate', to_timestamp('user_joinDate','E MMM dd HH:mm:ss +0000 yyyy'))

# create a view to use spark SQL
tweets_selected.createOrReplaceTempView('tweetsT')

# put the data frame into cache

tweets_selected.cache()

Out[1]: DataFrame[date: timestamp, lang: string, source: string, tweet_id: string, user_name: string, user_lang: string, user_location: string, user_verified: boolean, user_followers: bigint, user_friends: bigint, user_joinDate: timestamp, retweet_user: string, reply_count: bigint, retweet_count: bigint, favorite_count: bigint, retweet_text: string, text: string]

Below code will create a rating dataframe storing average rating for each tweet you may need for question 5.

In [0]:
from pyspark.sql.functions import col, explode, split, instr, avg, isnull, when

#load afinn for sentiment analysis
afinn = spark.read.option("inferSchema", "true").option("header", "true").option("delimiter", '\t')\
 .csv("/FileStore/tables/utilities/afinn.txt")


tweets_withID=tweets_selected.filter(col('lang')=='en').select('tweet_id', 'text', explode(split('text', " ")).alias('word'))

rating=tweets_withID.join(afinn, tweets_withID['word']==afinn['Word'], 'inner').groupBy('tweet_id', 'text').agg(avg('rating').alias('rating'))

rating.printSchema()

root
-- tweet_id: string (nullable = true)
-- text: string (nullable = true)
-- rating: double (nullable = true)

1. what is the number of **RETWEETS** by week day? Which day has the most retweets?

In [0]:
from pyspark.sql.functions import *
display(tweets_selected.groupBy(date_format('date', 'E').alias('Weekday')).agg(sum('retweet_count').alias("Total Retweets")).orderBy('Weekday', ascending = True))

Weekday,Total Retweets
Fri,1254934
Mon,177300
Sat,454104
Sun,392692
Thu,1382326
Tue,121395
Wed,295583


## Thursday has the most retweets

2.For all tweets in Japanese (ja), what are the top 5 locations of the users excluding null and Japan

In [0]:
from pyspark.sql.functions import *
display(tweets_selected.select('user_location').filter(col('lang') == 'ja').groupBy('user_location').count().alias('number of tweets').orderBy('count', ascending = False).na.drop().limit(5))

user_location,count
london,205
日本,76
Tokyo,62
牧之原市,45
神奈川県川崎市,43


## London is the top location with Japanese tweets

3.Find the total number of tweets where the text contains either charge 2 or charge 3 (please check page 96 of the your text for references)

In [0]:
from pyspark.sql.functions import *
tweets_selected.select('tweet_id').filter(instr(lower(col('text')), 'charge 2' or 'charge 3' )>=1).count()

Out[233]: 312

## There are 312 tweets about the Charge 2 or Charge 3 device

4.What is the percentage of verified users?

In [0]:
from pyspark.sql.functions import *
display(tweets_selected.select('user_verified').alias('Verified').groupBy('user_verified').count().orderBy('count', ascending=False))

user_verified,count
false,33020
true,3025


## 92% of users are NOT verified

5.What is average rating of tweets between verified users and non-verified users?

In [0]:
inner_join = tweets_selected.join(rating, tweets_selected.tweet_id == rating.tweet_id)
display(inner_join)

date,lang,source,tweet_id,user_name,user_lang,user_location,user_verified,user_followers,user_friends,user_joinDate,retweet_user,reply_count,retweet_count,favorite_count,retweet_text,text,tweet_id,text,rating
2019-10-01T04:13:53.000+0000,en,Twitter for iPhone,1178885721697837056,fred_dog,null,"Los Angeles, CA",false,915,1009,2009-05-08T16:24:50.000+0000,null,null,null,null,null,@KamiSawZe It looks like I may be going that way. Although I like the battery life of the fitbit and that I can wea… https://t.co/qYlW3jVReH,1178885721697837056,@KamiSawZe It looks like I may be going that way. Although I like the battery life of the fitbit and that I can wea… https://t.co/qYlW3jVReH,2.0
2019-10-01T09:52:20.000+0000,en,Twitter for iPhone,1178970895232258048,TXFunGuy6,null,In your wildest dreams,false,2496,2806,2013-01-02T05:25:13.000+0000,RunOldMan,12,47,135,"I'm going to bring a lawsuit against fitbit, I've been wearing this thing for five years and I'm still a shlumpy.","RT @RunOldMan: I'm going to bring a lawsuit against fitbit, I've been wearing this thing for five years and I'm still a shlumpy.",1178970895232258048,"RT @RunOldMan: I'm going to bring a lawsuit against fitbit, I've been wearing this thing for five years and I'm still a shlumpy.",-2.0
2019-10-01T12:36:38.000+0000,en,Twitter Web App,1179012243293396992,BigUncT,null,"In The Pocket, USA",false,266,458,2011-12-22T21:07:09.000+0000,null,null,null,null,null,I refuse to get a @fitbit until it can tell me how long in inches each shast I drop comes in at. fecal measurements speak volumes,1179012243293396992,I refuse to get a @fitbit until it can tell me how long in inches each shast I drop comes in at. fecal measurements speak volumes,-1.5
2019-10-01T13:13:35.000+0000,en,Twitter for iPhone,1179021544091193344,DeGolierThomas,null,Texas,false,1513,4062,2012-07-03T19:43:30.000+0000,null,null,null,null,null,To be clear my max was around 12 pounds higher than what I’ve lost this month. I was just too embarrassed to log th… https://t.co/OoCELSPn75,1179021544091193344,To be clear my max was around 12 pounds higher than what I’ve lost this month. I was just too embarrassed to log th… https://t.co/OoCELSPn75,-1.3333333333333333
2019-10-01T20:30:53.000+0000,en,Twitter Web App,1179131591521636353,billm75,null,"Hopkins, MN",false,4,37,2009-04-25T20:50:51.000+0000,null,null,null,null,null,@FitbitSupport I'm having trouble with the Fitbit app for android it telling me inaccurate steps per day like today… https://t.co/wOsG2TZQ97,1179131591521636353,@FitbitSupport I'm having trouble with the Fitbit app for android it telling me inaccurate steps per day like today… https://t.co/wOsG2TZQ97,0.0
2019-10-02T03:53:18.000+0000,en,Twitter for iPhone,1179242928214020098,_DonriELLE_,null,null,false,89,67,2012-06-13T06:58:49.000+0000,null,null,null,null,null,If you don’t want to read: go to bed at the same time every night.,1179242928214020098,If you don’t want to read: go to bed at the same time every night.,1.0
2019-10-02T04:08:09.000+0000,en,Twitter Web App,1179246666198765569,Northerngent4,null,null,false,57,194,2019-07-17T20:36:35.000+0000,null,null,null,null,null,"I’m currently staying up late to charge my Fitbit, so it won’t die and track my sleep patterns without dying. It… https://t.co/PC9rxjJl5B",1179246666198765569,"I’m currently staying up late to charge my Fitbit, so it won’t die and track my sleep patterns without dying. It… https://t.co/PC9rxjJl5B",-3.0
2019-10-02T04:17:12.000+0000,en,Blog2Social APP,1179248945446436865,city_republik,null,Ghana,false,7,25,2019-05-10T09:58:47.000+0000,null,null,null,null,null,"This article originally appeared on VICE UK.Your dad is in love with his FitBit, that girl at work has a borderline… https://t.co/ITVzzol4ti",1179248945446436865,"This article originally appeared on VICE UK.Your dad is in love with his FitBit, that girl at work has a borderline… https://t.co/ITVzzol4ti",3.0
2019-10-02T05:15:30.000+0000,en,Twitter Web App,1179263616601743362,pmkoom,nu

In [0]:
from pyspark.sql.functions import *
display(inner_join.groupBy('user_verified').agg(avg(rating['rating']).alias('Rating')))

user_verified,Rating
true,1.113579566335906
false,1.174564097215785


## There is not a strong difference in sentiment between verified and non verified users

# Lets add in a rating category

In [0]:
addedratings = inner_join.withColumn('sentiment', when(col('rating')>=1, 'positive').when(col('rating')<=-1, 'Negative').otherwise('neutral')).orderBy('rating', ascending = True)

In [0]:
display(addedratings)

date,lang,source,tweet_id,user_name,user_lang,user_location,user_verified,user_followers,user_friends,user_joinDate,retweet_user,reply_count,retweet_count,favorite_count,retweet_text,text,tweet_id,text,rating,sentiment
2019-10-10T03:17:16.000+0000,en,Twitter for iPhone,1182132965100412928,FunInFishers,null,"Fishers, IN",false,1720,180,2016-12-20T19:33:30.000+0000,null,null,null,null,null,@NotYourDaddy81 I wonder how many calories her @fitbit says she burned sucking cock and getting a facial?,1182132965100412928,@NotYourDaddy81 I wonder how many calories her @fitbit says she burned sucking cock and getting a facial?,-5.0,Negative
2019-10-01T13:54:38.000+0000,en,Twitter for iPhone,1179031872782786560,lucianlibrarian,null,"Florida, USA",false,17,98,2019-09-07T01:29:52.000+0000,null,null,null,null,null,@ignisgayentia It’s still loading because my Fitbit is being a bitch,1179031872782786560,@ignisgayentia It’s still loading because my Fitbit is being a bitch,-5.0,Negative
2019-10-10T03:04:43.000+0000,en,Twitter for Android,1182129807775145985,Pig_Minted,null,Under your toenail,false,685,170,2013-07-30T03:01:56.000+0000,null,null,null,null,null,Using a Fitbit as a cock ring so I can get my strokes per minute,1182129807775145985,Using a Fitbit as a cock ring so I can get my strokes per minute,-5.0,Negative
2019-10-03T13:58:42.000+0000,en,Twitter for iPhone,1179757673094483970,Eimearoneill16,null,Clonoe,false,272,520,2012-02-23T12:26:51.000+0000,null,null,null,null,null,my fitbit definitely thinks i’m a fat bastard cos it tells me to move every 5 seconds,1179757673094483970,my fitbit definitely thinks i’m a fat bastard cos it tells me to move every 5 seconds,-5.0,Negative
2019-10-30T19:22:21.000+0000,en,Twitter for iPhone,1189623591693717504,FatSportsGod,null,null,false,5,4,2019-10-20T20:31:18.000+0000,null,null,null,null,null,shut up bitch,1189623591693717504,shut up bitch,-5.0,Negative
2019-10-10T18:01:10.000+0000,en,Twitter for Android,1182355406468849666,moonageblues,null,null,false,842,415,2013-05-11T12:50:42.000+0000,null,null,null,null,null,i met an 8 year old with an iphone and a fitbit. hate to be that bitch but wtf,1182355406468849666,i met an 8 year old with an iphone and a fitbit. hate to be that bitch but wtf,-4.0,Negative
2019-10-04T22:42:50.000+0000,en,Twitter for iPhone,1180251962018861056,fairyleo_,null,"Lexington, KY",false,383,908,2012-05-28T19:51:11.000+0000,null,null,null,null,null,Fitbit just sent me an email that some “suspicious activity” has happened on my account... Oh shit someone else is… https://t.co/gm94lVnkqq,1180251962018861056,Fitbit just sent me an email that some “suspicious activity” has happened on my account... Oh shit someone else is… https://t.co/gm94lVnkqq,-4.0,Negative
2019-10-11T01:46:35.000+0000,en,Twitter for Android,1182472532697518080,zimmermanncharm,null,"Toronto, Ontario",false,186,391,2016-11-19T01:48:40.000+0000,null,null,null,null,null,The Leafs pissed me off enough that I left for work early and forgot my Fitbit and moisturizer at home.,1182472532697518080,The Leafs pissed me off enough that I left for work early and forgot my Fitbit and moisturizer at home.,-4.0,Negative
2019-10-06T12:27:03.000+0000,en,Twitter for iPhone,1180821772796661760,xEmeraldPhoenix,null,#Bregan: 11/19/18 🖤,false,2054,1459,2009-09-23T17:05:27.000+0000,null,null,null,null,null,How about no. This is fucking FitBit. Fuck off. https://t.co/kzxdCpy0rq,1180821772796661760,How about no. This is fucking FitBit. Fuck off. https://t.co/kzxdCpy0rq,-4.0,Negative
2019-10-03T20:23:58.000+0000,en,Twitter for iPhone,1179854626562232326,meerihaataja,null,Helsinki - New York,false,3152,2845,2010-03-02T14:48:04.000+0000,null,null,null,null,null,"@hgavert @ruchowdh @iajunwa Hugo I know what you talk about and at times, I feel the same. Today I’m pissed as my F… https://t.co/JYV4PwyrPs",1179854626562232326,"@hgavert @ruchowdh @iajunwa Hugo I know what you talk about and at times, I feel the same. Today I’m pissed as my F… https:

## Lets count the number of tweets by location and category

In [0]:
display(addedratings.select('user_location', 'sentiment').groupBy('user_location', 'sentiment').count().alias('number of tweets').orderBy('count', ascending = False).na.drop().limit(5))

user_location,sentiment,count
london,positive,217
United Kingdom,positive,166
UK,positive,143
"London, England",positive,120
"England, United Kingdom",positive,107


## London has the most tweets by location and the data shows us that most of the tweets coming from London are positive